In [21]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/nitinsinghmehta16may/10-MLOPS-Project-1.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="nitinsinghmehta16may"
os.environ["MLFLOW_TRACKING_PASSWORD"]="28e4961c5373bc3e6b22c843b023b899f840f5e7"

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
MLFLOW_TRACKING_URI=os.getenv("MLFLOW_TRACKING_URI")
MLFLOW_TRACKING_USERNAME=os.getenv("MLFLOW_TRACKING_USERNAME")
MLFLOW_TRACKING_PASSWORD=os.getenv("MLFLOW_TRACKING_PASSWORD")

In [ ]:
# print(MLFLOW_TRACKING_PASSWORD)
# print(MLFLOW_TRACKING_URI)

28e4961c5373bc3e6b22c843b023b899f840f5e7
https://dagshub.com/nitinsinghmehta16may/10-MLOPS-Project-1.mlflow


In [23]:
import os
%pwd

'c:\\Users\\nitin\\DSML\\100.MLOPS\\10-MLOPS-Project-1'

In [24]:
# os.chdir("../")
# %pwd

In [25]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [26]:
from src.DataScience.constants import *
from src.DataScience.utils.common import read_yaml, create_directories,save_json

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifact_dir])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri= MLFLOW_TRACKING_URI
        )
        return model_evaluation_config


In [28]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [29]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    


In [30]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

Trying to load YAML from: c:\Users\nitin\DSML\100.MLOPS\10-MLOPS-Project-1\config\config.yaml
2025-10-01 10:24:38,221: INFO: src.DataScience: yaml file: c:\Users\nitin\DSML\100.MLOPS\10-MLOPS-Project-1\config\config.yaml loaded successfully
Trying to load YAML from: c:\Users\nitin\DSML\100.MLOPS\10-MLOPS-Project-1\params.yaml
2025-10-01 10:24:38,224: INFO: src.DataScience: yaml file: c:\Users\nitin\DSML\100.MLOPS\10-MLOPS-Project-1\params.yaml loaded successfully


Trying to load YAML from: c:\Users\nitin\DSML\100.MLOPS\10-MLOPS-Project-1\schema.yaml
2025-10-01 10:24:38,233: INFO: src.DataScience: yaml file: c:\Users\nitin\DSML\100.MLOPS\10-MLOPS-Project-1\schema.yaml loaded successfully
2025-10-01 10:24:38,235: INFO: src.DataScience: created directory at: artifact
2025-10-01 10:24:38,238: INFO: src.DataScience: created directory at: artifact/model_evaluation


c:\Users\nitin\anaconda3\envs\venv-MLOPS\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator ElasticNet from version 1.5.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


2025-10-01 10:24:38,755: INFO: src.DataScience: json file saved at: artifact\model_evaluation\metrics.json


2025/10/01 10:24:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2025/10/01 10:25:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


🏃 View run honorable-fowl-568 at: https://dagshub.com/nitinsinghmehta16may/10-MLOPS-Project-1.mlflow/#/experiments/0/runs/8c8f1ff7c13a4c81bed40fd2d5efd702
🧪 View experiment at: https://dagshub.com/nitinsinghmehta16may/10-MLOPS-Project-1.mlflow/#/experiments/0
